In [1]:
import pandas as pd
import neattext.functions as nfx

In [2]:
import numpy as np
import json
import glob

#Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

from gensim.parsing.preprocessing import preprocess_string, STOPWORDS, RE_TAGS, RE_PUNCT, RE_WHITESPACE

#spacy
import spacy
import nltk
# nltk.download()
from nltk.corpus import stopwords

#vis
import pyLDAvis
import pyLDAvis.gensim_models

import warnings
warnings.filterwarnings("ignore")

pyLDAvis.enable_notebook()

/Users/annetteh/opt/anaconda3/lib/python3.8/site-packages/sklearn/decomposition/_lda.py:28: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  EPS = np.finfo(np.float).eps


In [3]:
online = pd.read_csv('assets/original/2021-10-19-MichiganOnline-courses.csv')
f_21 = pd.read_csv('assets/f_21_merge.csv')
w_22 = pd.read_csv('assets/w_22_merge.csv')

In [4]:
online['description']

0      Through this course, you will start by address...
1      The third and the last course of the Addressin...
2      Are you concerned about climate change? Would ...
3      This course, Additive Manufacturing, is the th...
4      This course builds upon the fundamental concep...
                             ...                        
516    This module examines the impacts of incarcerat...
517    This third course in the “Good with Words: Wri...
518    This fourth and final course in the “Good with...
519    This second course in the Good with Words: Wri...
520    This course will teach you how to use your wri...
Name: description, Length: 521, dtype: object

In [5]:
w_22['description']

0        This course seeks to introduce students to eve...
1        This seminar introduces first-year students to...
2        This class explores the fascinating shift in B...
3        This seminar introduces first-year students to...
4        This seminar introduces first-year students to...
                               ...                        
66693                                                  NaN
66694                                                  NaN
66695                                                  NaN
66696                                                  NaN
66697                                                  NaN
Name: description, Length: 66698, dtype: object

In [66]:
def clean_tokenize_text(tweet_df):

    tweet_df['description'] = tweet_df['description'].astype(str)
    
    tweet_df['tokens'] = [word for word in tweet_df['description'].str.split()]



    STOPWORDS_list = [x for x in STOPWORDS]

    def remove_stopwords(tokens, stopwords=custom_stop_words + STOPWORDS_list):
        return [token for token in tokens if token not in stopwords]

    tweet_df['tokens'] = tweet_df['tokens'].apply(lambda x: remove_stopwords(x))
    
    tweet_df['tokens'] = tweet_df['tokens'].apply(lambda x: preprocess_string(x))
    
    return tweet_df

In [67]:
clean_tokenize_text(w_22)

TypeError: decoding to str: need a bytes-like object, list found

In [20]:
from collections import Counter, defaultdict

In [24]:
def lda_model_for_each_df(df, num_topics):
    
    data = df['description'].dropna()
    
    def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
        nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
        texts_out = []
        for text in texts:
            doc = nlp(text)
            new_text = []
            for token in doc:
                if token.pos_ in allowed_postags:
                    new_text.append(token.lemma_)
            final = " ".join(new_text)
            texts_out.append(final)
            
        return texts_out
    
    lemmatized_texts = lemmatization(data)
    
    def gen_words(texts):
        final = []
        for text in texts:
            new = gensim.utils.simple_preprocess(text, deacc=True)
            final.append(new)
        return final
    
    

    data_words = gen_words(lemmatized_texts)
    
    common_words = Counter([item for sublist in data_words for item in sublist]).most_common(200)
    
    custom_stop_words = [word_count[0] for word_count in common_words]
    
    return custom_stop_words

In [26]:
data_words = lda_model_for_each_df(w_22, 10)

In [68]:
bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

bigram = gensim.models.phrases.Phraser(bigram_phrases)
trigram = gensim.models.phrases.Phraser(trigram_phrases)

def make_bigrams(texts):
    return([bigram[doc] for doc in texts])

def make_trigrams(texts):
    return ([trigram[bigram[doc]] for doc in texts])

data_bigrams = make_bigrams(data_words)
data_bigrams_trigrams = make_trigrams(data_bigrams)


# TF-IDF removal

from gensim.models import TfidfModel

id2word = corpora.Dictionary(data_bigrams_trigrams)

texts = data_bigrams_trigrams

corpus = [id2word.doc2bow(text) for text in texts]

tfidf = TfidfModel(corpus, id2word=id2word)

In [76]:
for sublist in data_words:
    sublist = [item for item in sublist if item in custom_stop_words]

In [85]:
data_words

[['course',
  'seek',
  'introduce',
  'student',
  'everyday',
  'life',
  'urban',
  'course',
  'be',
  'design',
  'equip',
  'student',
  'basic',
  'useful',
  'knowledge',
  'how',
  'urban',
  'resident',
  'rich',
  'poor',
  'newcomer',
  'old',
  'timer',
  'young',
  'old',
  'man',
  'woman',
  'negotiate',
  'challenge',
  'live',
  'city',
  'course',
  'focus',
  'network',
  'associational',
  'life',
  'relationship',
  'be',
  'tie',
  'bind',
  'urban',
  'resident',
  'together',
  'social',
  'organization',
  'religious',
  'belief',
  'practice',
  'ethnicity',
  'economic',
  'political',
  'system',
  'art',
  'popular',
  'culture',
  'be',
  'topic',
  'explore',
  'be',
  'approach',
  'theme',
  'variety',
  'disciplinary',
  'perspective',
  'include',
  'history',
  'anthropology',
  'literature',
  'political',
  'science',
  'sociology',
  'economic'],
 ['seminar',
  'introduce',
  'first',
  'year',
  'student',
  'intellectual',
  'community',
  'hum

In [83]:
[item for sublist in data_words for item in sublist if item not in custom_stop_words]

['seek',
 'everyday',
 'urban',
 'equip',
 'basic',
 'useful',
 'knowledge',
 'urban',
 'resident',
 'rich',
 'poor',
 'newcomer',
 'old',
 'timer',
 'young',
 'old',
 'man',
 'woman',
 'negotiate',
 'challenge',
 'live',
 'city',
 'network',
 'associational',
 'relationship',
 'tie',
 'bind',
 'urban',
 'resident',
 'together',
 'organization',
 'religious',
 'belief',
 'ethnicity',
 'economic',
 'political',
 'system',
 'art',
 'popular',
 'theme',
 'disciplinary',
 'perspective',
 'anthropology',
 'political',
 'sociology',
 'economic',
 'seminar',
 'first',
 'year',
 'intellectual',
 'community',
 'humanity',
 'scholar',
 'afroamerican',
 'african',
 'seminar',
 'religious',
 'belief',
 'institution',
 'dialogue',
 'african',
 'african',
 'origin',
 'community',
 'atlantic',
 'transnational',
 'relationship',
 'black',
 'community',
 'different',
 'examine',
 'religion',
 'inform',
 'slavery',
 'identity',
 'colonialism',
 'revolution',
 'nationalism',
 'globalization',
 'popular',

In [19]:
def lda_model_for_each_df(df, num_topics):
    
    data = df['description'].dropna()
    
    def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
        nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
        texts_out = []
        for text in texts:
            doc = nlp(text)
            new_text = []
            for token in doc:
                if token.pos_ in allowed_postags:
                    new_text.append(token.lemma_)
            final = " ".join(new_text)
            texts_out.append(final)
        return (texts_out)

    lemmatized_texts = lemmatization(data)

    def gen_words(texts):
        final = []
        for text in texts:
            new = gensim.utils.simple_preprocess(text, deacc=True)
            final.append(new)
        return (final)

    data_words = gen_words(lemmatized_texts)


    # Create bigrams and trigrams
    
    bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)

    def make_bigrams(texts):
        return([bigram[doc] for doc in texts])

    def make_trigrams(texts):
        return ([trigram[bigram[doc]] for doc in texts])

    data_bigrams = make_bigrams(data_words)
    data_bigrams_trigrams = make_trigrams(data_bigrams)


    # TF-IDF removal
    
    from gensim.models import TfidfModel

    id2word = corpora.Dictionary(data_bigrams_trigrams)

    texts = data_bigrams_trigrams

    corpus = [id2word.doc2bow(text) for text in texts]

    tfidf = TfidfModel(corpus, id2word=id2word)

    low_value = 0.05
    words  = []
    words_missing_in_tfidf = []
    for i in range(0, len(corpus)):
        bow = corpus[i]
        low_value_words = []
        tfidf_ids = [id for id, value in tfidf[bow]]
        bow_ids = [id for id, value in bow]
        low_value_words = [id for id, value in tfidf[bow] if value < low_value]
        drops = low_value_words + words_missing_in_tfidf
        for item in drops:
            words.append(id2word[item])
        words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
        corpus[i] = new_bow


    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                               id2word=id2word,
                                               num_topics=num_topics,
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha="auto")

    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)

    return vis

In [78]:
top_100 = Counter([item for sublist in data_words for item in sublist]).most_common(200)

In [79]:
custom_stop_words = [i[0] for i in top_100]

In [19]:
def lda_model_for_each_df(df, num_topics):
    
    data = df['description'].dropna()
    
    def lemmatization(texts, allowed_postags=["NOUN", "ADJ", "VERB", "ADV"]):
        nlp = spacy.load("en_core_web_sm", disable=["parser", "ner"])
        texts_out = []
        for text in texts:
            doc = nlp(text)
            new_text = []
            for token in doc:
                if token.pos_ in allowed_postags:
                    new_text.append(token.lemma_)
            final = " ".join(new_text)
            texts_out.append(final)
        return (texts_out)

    lemmatized_texts = lemmatization(data)

    def gen_words(texts):
        final = []
        for text in texts:
            new = gensim.utils.simple_preprocess(text, deacc=True)
            final.append(new)
        return (final)

    data_words = gen_words(lemmatized_texts)


    # Create bigrams and trigrams
    
    bigram_phrases = gensim.models.Phrases(data_words, min_count=5, threshold=50)
    trigram_phrases = gensim.models.Phrases(bigram_phrases[data_words], threshold=50)

    bigram = gensim.models.phrases.Phraser(bigram_phrases)
    trigram = gensim.models.phrases.Phraser(trigram_phrases)

    def make_bigrams(texts):
        return([bigram[doc] for doc in texts])

    def make_trigrams(texts):
        return ([trigram[bigram[doc]] for doc in texts])

    data_bigrams = make_bigrams(data_words)
    data_bigrams_trigrams = make_trigrams(data_bigrams)


    # TF-IDF removal
    
    from gensim.models import TfidfModel

    id2word = corpora.Dictionary(data_bigrams_trigrams)

    texts = data_bigrams_trigrams

    corpus = [id2word.doc2bow(text) for text in texts]

    tfidf = TfidfModel(corpus, id2word=id2word)

    low_value = 0.05
    words  = []
    words_missing_in_tfidf = []
    for i in range(0, len(corpus)):
        bow = corpus[i]
        low_value_words = []
        tfidf_ids = [id for id, value in tfidf[bow]]
        bow_ids = [id for id, value in bow]
        low_value_words = [id for id, value in tfidf[bow] if value < low_value]
        drops = low_value_words + words_missing_in_tfidf
        for item in drops:
            words.append(id2word[item])
        words_missing_in_tfidf = [id for id in bow_ids if id not in tfidf_ids]

        new_bow = [b for b in bow if b[0] not in low_value_words and b[0] not in words_missing_in_tfidf]
        corpus[i] = new_bow


    lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus[:-1],
                                               id2word=id2word,
                                               num_topics=num_topics,
                                               random_state=100,
                                               update_every=1,
                                               chunksize=100,
                                               passes=10,
                                               alpha="auto")

    vis = pyLDAvis.gensim_models.prepare(lda_model, corpus, id2word, mds="mmds", R=10)

    return vis

In [20]:
lda_model_for_each_df(f_21, 30)

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
20    -0.510270  0.123330       1        1  12.753857
26     0.128530 -0.516415       2        1   8.506688
13     0.469772  0.220214       3        1   8.248007
17     0.365838  0.344131       4        1   5.773676
29     0.488302 -0.116539       5        1   5.736242
3     -0.034689 -0.448339       6        1   5.479134
16    -0.297153  0.434608       7        1   5.382717
10     0.489166  0.053283       8        1   4.610392
1     -0.450661 -0.259238       9        1   3.903377
21    -0.497220 -0.048864      10        1   3.885254
24    -0.040211  0.404735      11        1   3.875115
15    -0.177449 -0.460608      12        1   3.722815
23    -0.291762 -0.364330      13        1   2.864548
22     0.298132 -0.044190      14        1   2.593297
14    -0.040002  0.485506      15        1   2.574925
4      0.404344 -0.258433      16        1   2.475863
18     0.299502 -0.298029      17        1   2.322749
9     -0.400817  0.263199      18        1   2.263934
5     -0.345058 -0.163969      19        1   2.082458
7      0.150297  0.465590      20        1   1.851401
2      0.199334  0.319310      21        1   1.673213
19    -0.246412  0.140984      22        1   1.478801
11    -0.140168  0.263522      23        1   1.427852
8      0.192647 -0.372316      24        1   1.400387
12     0.193935  0.125488      25        1   0.915104
28    -0.110757 -0.154858      26        1   0.886981
0      0.043790 -0.223404      27        1   0.885649
6     -0.136791  0.019026      28        1   0.401334
25    -0.002084  0.033302      29        1   0.012115
27    -0.002084  0.033303      30        1   0.012114, topic_info=          Term          Freq         Total Category  logprob  loglift
5           be  34714.000000  34714.000000  Default   10.000  10.0000
53       topic  12128.000000  12128.000000  Default    9.000   9.0000
47     student  17772.000000  17772.000000  Default    8.000   8.0000
289   activity  17705.000000  17705.000000  Default    7.000   7.0000
288      write  15273.000000  15273.000000  Default    6.000   6.0000
..         ...           ...           ...      ...      ...      ...
4        basic      0.009079   1550.765172  Topic30   -9.185  -3.0297
5           be      0.009079  34714.965270  Topic30   -9.185  -6.1381
6       belief      0.009079     53.105240  Topic30   -9.185   0.3445
8    challenge      0.009079   1114.780076  Topic30   -9.185  -2.6996
9         city      0.009079    404.194136  Topic30   -9.185  -1.6851

[354 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
360       5  0.999378   acquire
588      28  0.960689       act
496       7  0.997709    active
289       2  0.999862  activity
363       3  0.999659  advanced
...     ...       ...       ...
857      25  0.992366      wide
152      19  0.999545      work
172       8  0.999476     world
288       4  0.999838     write
153      22  0.999128      year

[240 rows x 3 columns], R=10, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[21, 27, 14, 18, 30, 4, 17, 11, 2, 22, 25, 16, 24, 23, 15, 5, 19, 10, 6, 8, 3, 20, 12, 9, 13, 29, 1, 7, 26, 28])